# TopicBank: Bank Creation Experiment

Dataset: [PostNauka](https://postnauka.ru/) articles.

In [ ]:
# General imports

import itertools
import numpy as np
import os
import pandas as pd

from scipy.stats import gaussian_kde
from matplotlib import pyplot as plt

%matplotlib inline

In [ ]:
# Making `topnum` module visible for Python

import sys

sys.path.insert(0, '..')

In [ ]:
# Optimal number of topics

from topicnet.cooking_machine import Dataset

from topnum.data.vowpal_wabbit_text_collection import VowpalWabbitTextCollection
from topnum.scores import (
    IntratextCoherenceScore,
    SparsityPhiScore,
    SparsityThetaScore,
    SimpleTopTokensCoherenceScore,
    SophisticatedTopTokensCoherenceScore,
)
from topnum.scores._base_coherence_score import (
    SpecificityEstimationMethod,
    TextType,
    WordTopicRelatednessType
)
from topnum.scores.intratext_coherence_score import ComputationMethod
from topnum.search_methods import TopicBankMethod
from topnum.search_methods.topic_bank.one_model_train_funcs import (
    default_train_func,
    regularization_train_func,
    specific_initial_phi_train_func,
    background_topics_train_func,
)


## Data

In the folder below must reside the necessary data file in .csv format.

In [ ]:
DATA_FOLDER_PATH = 'data'

In [ ]:
os.listdir(DATA_FOLDER_PATH)

['bigartm.miptai.vasiliyalekseev.log.INFO.20200322-195518.20211',
 '_dataset_rxg0krms',
 'bigartm.miptai.vasiliyalekseev.log.INFO.20200322-200334.22948',
 'postnauka__dataset__natural_order_batches',
 'postnauka__vw__natural_order.txt',
 'bigartm.miptai.vasiliyalekseev.log.INFO.20200322-195026.19312',
 'bigartm.miptai.vasiliyalekseev.log.INFO.20200322-195237.19407',
 'bigartm.miptai.vasiliyalekseev.log.INFO.20200322-200237.21203',
 '_dataset_lthzj9qc',
 '_dataset_ux1_6tj1',
 '_dataset_hvteto6c',
 '_dataset_1fzjutvk',
 'bigartm.miptai.vasiliyalekseev.log.INFO.20200322-195348.19573',
 '_dataset_0o0fiiqo',
 'postnauka__vocab.txt',
 '_dataset_a00ok3up',
 '_dataset_lh9rv2te',
 '_dataset_0jpn3owo',
 'cooc',
 '_dataset_m9qhqs6p',
 '_dataset_5qy6w9hv',
 'bigartm.INFO',
 'twenty_newsgroups__vw__natural_order.txt',
 'postnauka__dataset__natural_order.csv']

In [ ]:
dataset_file_name = 'postnauka__dataset__natural_order.csv'

In [ ]:
dataset_file_path = os.path.join(
    DATA_FOLDER_PATH,
    dataset_file_name
)

Checking if all OK with data, what modalities does the collection have.

In [ ]:
! head -n 2 $dataset_file_path

id,raw_text,vw_text
29998.txt,материал отрицательный показатель преломление физик виктор веселаго распространение свет вещество фазовый групповой скорость метаматериалы различаться фазовый групповой скорость каков физика распространение свет вещество находить применение материал отрицательный показатель преломление рассказывать доктор физикоматематический наука виктор веселаго скорость распространяться энергия вещество обычно говорить излучение распространяться вещество со скорость n раз маленький n коэффициент преломление вещество коэффициент преломление n отношение скорость свет скорость распространение излучение вещество обычно уточняться распространяться распространение энергия распространение импульс происходить различный закон энергия распространяться со скорость называться групповой скорость много скорость свет эйнштейн сформулировать самый больший скорость излучение скорость свет кмс импульс распространяться фазовый скорость сколь угодно много скорость свет скорость входить соо

In [ ]:
dataset = Dataset(dataset_file_path)

In [ ]:
dataset._data.shape

(3446, 2)

In [ ]:
dataset._data.head()

,raw_text,vw_text
id,,
29998.txt,материал отрицательный показатель преломление ...,29998.txt |@text материал отрицательный показа...
7770.txt,культурный код экономика экономист александр а...,7770.txt |@text культурный код экономика эконо...
32230.txt,faq наука третий класс факт эксперимент резуль...,32230.txt |@text faq наука третий класс факт э...
27293.txt,обрушение волна поверхность жидкость математик...,27293.txt |@text обрушение волна поверхность ж...
481.txt,существовать ли суперсимметрия мир элементарны...,481.txt |@text существовать ли суперсимметрия ...


In [ ]:
dataset.get_vw_document('29998.txt').loc['29998.txt', 'vw_text']

'29998.txt |@text материал отрицательный показатель преломление физик виктор веселаго распространение свет вещество фазовый групповой скорость метаматериалы различаться фазовый групповой скорость каков физика распространение свет вещество находить применение материал отрицательный показатель преломление рассказывать доктор физикоматематический наука виктор веселаго скорость распространяться энергия вещество обычно говорить излучение распространяться вещество со скорость n раз маленький n коэффициент преломление вещество коэффициент преломление n отношение скорость свет скорость распространение излучение вещество обычно уточняться распространяться распространение энергия распространение импульс происходить различный закон энергия распространяться со скорость называться групповой скорость много скорость свет эйнштейн сформулировать самый больший скорость излучение скорость свет кмс импульс распространяться фазовый скорость сколь угодно много скорость свет скорость входить соотношение emc

## Coocs

The notebook [Making-Decorrelation-and-Topic-Selection-Friends.ipynb](https://github.com/machine-intelligence-laboratory/TopicNet/blob/master/topicnet/demos/Making-Decorrelation-and-Topic-Selection-Friends.ipynb) contains a bit more explanation and references concerning cooccurrences computation in ARTM library.

In [ ]:
COOC_DATA_FOLDER_PATH = os.path.join(DATA_FOLDER_PATH, 'cooc')

In [ ]:
if os.path.isdir(COOC_DATA_FOLDER_PATH):
    print(os.listdir(COOC_DATA_FOLDER_PATH))

['cooc_tf_', 'cooc_values.pl', 'ppmi_df_', 'new_ppmi_tf_', 'ppmi_tf_', 'cooc_df_', 'cooc_values.json']


In [ ]:
cooc_values_file_path = os.path.join(
    DATA_FOLDER_PATH,
    'cooc',
    'cooc_values.json'
)

In [ ]:
if os.path.isfile(cooc_values_file_path):
    print(
        json.loads(open(cooc_values_file_path, 'r').read())[:20]
    )

[[['политический', 'специалист'], 0.0316201], [['специалист', 'политический'], 0.0316201], [['политический', 'реформа'], 1.32313], [['реформа', 'политический'], 1.32313], [['политический', 'развитый'], 0.021579], [['развитый', 'политический'], 0.021579], [['политический', 'разрешать'], 0.097536], [['разрешать', 'политический'], 0.097536], [['политический', 'попрежнему'], 0.442749], [['попрежнему', 'политический'], 0.442749], [['политический', 'попытаться'], 0.271374], [['попытаться', 'политический'], 0.271374], [['политический', 'гражданский'], 1.4379], [['гражданский', 'политический'], 1.4379], [['политический', 'группа'], 0.28741], [['группа', 'политический'], 0.28741], [['политический', 'чисто'], 0.764525], [['чисто', 'политический'], 0.764525], [['политический', 'представитель'], 0.685384], [['представитель', 'политический'], 0.685384]]


In [ ]:
if not os.path.isfile(cooc_values_file_path):
    cooc_values = dict()
else:
    raw_cooc_values = json.loads(open(cooc_values_file_path, 'r').read())

    cooc_values = {
        tuple(d[0]): d[1] for d in saved_raw_cooc_values
    }

In [ ]:
len(list(cooc_values.items()))

2152022

In [ ]:
print(list(cooc_values.items())[:10])

[(('политический', 'специалист'), 0.0316201), (('специалист', 'политический'), 0.0316201), (('политический', 'реформа'), 1.32313), (('реформа', 'политический'), 1.32313), (('политический', 'развитый'), 0.021579), (('развитый', 'политический'), 0.021579), (('политический', 'разрешать'), 0.097536), (('разрешать', 'политический'), 0.097536), (('политический', 'попрежнему'), 0.442749), (('попрежнему', 'политический'), 0.442749)]


## Scores (for Topics and Models)

In [ ]:
WINDOW = 20
NUM_TOP_WORDS = 20
MAX_NUM_OUT_WORDS = 5

In [ ]:
# Default scores in Topic Bank

main_topic_score = IntratextCoherenceScore(
    name='intratext_coherence_score__tt_vw__cm_seg_weight__wtrt_pwt__sem_none',
    data=dataset,
    text_type=TextType.VW_TEXT,
    computation_method=ComputationMethod.SEGMENT_WEIGHT,
    word_topic_relatedness=WordTopicRelatednessType.PWT,
    specificity_estimation=SpecificityEstimationMethod.NONE,
    max_num_out_of_topic_words=MAX_NUM_OUT_WORDS,
    window=WINDOW
)

other_topic_scores = [
    SophisticatedTopTokensCoherenceScore(
        name='top_tokens_coherence_score__tt_vw__wtrt_pwt__sem_none',
        data=dataset,
        text_type=TextType.VW_TEXT,
        word_topic_relatedness=WordTopicRelatednessType.PWT,
        specificity_estimation=SpecificityEstimationMethod.NONE,
        num_top_words=NUM_TOP_WORDS,
        window=WINDOW
    )
]

In [ ]:
# Other coherence scores variations

text_type_ids = {
    TextType.VW_TEXT: 'vw',
}
computation_method_ids = {
    ComputationMethod.SEGMENT_WEIGHT: 'seg_weight',
    ComputationMethod.SEGMENT_LENGTH: 'seg_length',
    ComputationMethod.SUM_OVER_WINDOW: 'sow',
}
word_topic_relatedness_type_ids = {
    WordTopicRelatednessType.PWT: 'pwt',
    WordTopicRelatednessType.PTW: 'ptw',
}
specificity_estimation_method_ids = {
    SpecificityEstimationMethod.NONE: 'none',
    SpecificityEstimationMethod.AVERAGE: 'av',
    SpecificityEstimationMethod.MAXIMUM: 'max',
}


param_combinations_intratext = list(
    itertools.product(
        text_type_ids,
        computation_method_ids,
        word_topic_relatedness_type_ids,
        specificity_estimation_method_ids,
    )
)
param_combinations_intratext.remove(
    (
        TextType.VW_TEXT,
        ComputationMethod.SEGMENT_WEIGHT,
        WordTopicRelatednessType.PWT,
        SpecificityEstimationMethod.NONE
    )
)

param_combinations_top_tokens = list(
    itertools.product(
        text_type_ids,
        word_topic_relatedness_type_ids,
        specificity_estimation_method_ids,
    )
)
param_combinations_top_tokens.remove(
    (
        TextType.VW_TEXT,
        WordTopicRelatednessType.PWT,
        SpecificityEstimationMethod.NONE
    )
)


for param_combination in param_combinations_intratext:
    (text_type,
     computation_method,
     word_topic_relatedness,
     specificity_estimation) = param_combination

    name = (
        f'intratext_coherence_score'
        f'__tt_{text_type_ids[text_type]}'
        f'__cm_{computation_method_ids[computation_method]}'
        f'__wtrt_{word_topic_relatedness_type_ids[word_topic_relatedness]}'
        f'__sem_{specificity_estimation_method_ids[specificity_estimation]}'
    )

    other_topic_scores.append(
        IntratextCoherenceScore(
            name=name,
            data=dataset,
            text_type=text_type,
            computation_method=computation_method,
            word_topic_relatedness=word_topic_relatedness,
            specificity_estimation=specificity_estimation,
            max_num_out_of_topic_words=MAX_NUM_OUT_WORDS,
            window=WINDOW
        )
    )


for param_combination in param_combinations_top_tokens:
    (text_type,
     word_topic_relatedness,
     specificity_estimation) = param_combination

    name = (
        f'top_tokens_coherence_score'
        f'__tt_{text_type_ids[text_type]}'
        f'__wtrt_{word_topic_relatedness_type_ids[word_topic_relatedness]}'
        f'__sem_{specificity_estimation_method_ids[specificity_estimation]}'
    )

    other_topic_scores.append(
        SophisticatedTopTokensCoherenceScore(
            name=name,
            data=dataset,
            text_type=text_type,
            word_topic_relatedness=word_topic_relatedness,
            specificity_estimation=specificity_estimation,
            num_top_words=NUM_TOP_WORDS,
            window=WINDOW
        )
    )

In [ ]:
# Another implementation of top-tokens-based coherence

param_combinations_other_top_tokens = list(
    product([True, False], ['median', 'mean'], [None, 1e-7])
)

if len(cooc_values) > 0:  # with pre-computed coocs
    for param_combination in param_combinations_other_top_tokens:
        (kernel,
         average,
         active_topic_threshold) = param_combination

        name = (
            f'top_tokens_coherence_other_implementation_score'
            f'__ker_{kernel}'
            f'__av_{average}'
            f'__att_{active_topic_threshold}'
        )

        other_topic_scores.append(
            SimpleTopTokensCoherenceScore(
                name=name,
                data=dataset,
                cooccurrence_values=cooc_values,
                num_top_tokens=20,
                kernel=kernel,
                average=average,
                active_topic_threshold=active_topic_threshold,
            )
        )

In [ ]:
# Default scores in Topic Bank

other_scores = [
    SparsityPhiScore(
        name='sparsity_phi_score'
    ),
    SparsityThetaScore(
        name='sparsity_theta_score'
    ),
]

## Bank Creation

In [ ]:
# Default train func

train_funcs = default_train_func

In [ ]:
# TODO: use Holdout Perplexity as Stop score

optimizer = TopicBankMethod(
    data = dataset,
    min_df_rate = 0.025,
    max_df_rate = 0.8,
    
    main_topic_score = main_topic_score,
    other_topic_scores = other_topic_scores,
    other_scores = other_scores,
    
    max_num_models = 5,
    one_model_num_topics = 5,
    num_fit_iterations = 2,
    topic_score_threshold_percentile = 90,
    
    train_funcs = train_funcs,
)

Fulfilling the search:

In [ ]:
%%time

optimizer.search_for_optimum(dataset)

In [ ]:
! ls $optimizer._topic_bank._path

## Appendix. Making vocab.txt for Computing Coocs

In [ ]:
dataset._cached_dict = None  # TODO: bug in TopicNet

dictionary = dataset.get_dictionary()
dictionary.filter(min_df_rate=0.025, max_df_rate=0.8)

dictionary_file_path = os.path.join(
    DATA_FOLDER_PATH,
    'dict.dict',
)

dictionary.save_text(dictionary_file_path)

/home/vasiliyalekseev/miniconda/lib/python3.7/site-packages/topicnet/cooking_machine/dataset.py:371: UserWarning: Attempted to use batches for different dataset.Overwriting batches in data/postnauka__dataset__natural_order_batches
  warnings.warn(W_DIFF_BATCHES_1 + W_DIFF_BATCHES_2.format(batch_vectorizer_path))


In [ ]:
lines = open(dictionary_file_path, 'r').readlines()

In [ ]:
len(lines)

2516

In [ ]:
lines[:3]

['name: 1f56d8b1-1e68-42a4-b975-9713b1546b85 num_items: 3446\n',
 'token, class_id, token_value, token_tf, token_df\n',
 'политический, @text, 0.0012545293429866433, 2802.0, 649.0\n']

In [ ]:
lines[-3:]

['построить, @text, 0.0003604197409003973, 805.0, 516.0\n',
 'ничто, @text, 0.0006035351543687284, 1348.0, 782.0\n',
 'различать, @text, 0.00010655888036126271, 238.0, 185.0\n']

In [ ]:
vocab_text = ''

for line in lines[2:]:
    token, modality, _, _, _ = line.strip().split(', ')
    vocab_text += f'{token} {modality}\n'

In [ ]:
vocab_file_path = os.path.join(
    DATA_FOLDER_PATH,
    'postnauka__vocab.txt',
)

In [ ]:
with open(vocab_file_path, 'w') as f:
    f.write(vocab_text)